In [ ]:
### LASSO estimation of entire set or a selected model
# initialise the Lasso CV model
lasso_reg(x[['unempl', 'Ind_Prod', 'Term', 'Curve']],y)

x_scale = x[['unempl', 'Ind_Prod', 'Term', 'Curve']].std(ddof=0)
y_scale = y.std(ddof=0)
std_x = x / x_scale
std_y = y / y_scale
std_x = std_x[['unempl', 'Ind_Prod', 'Term', 'Curve']]
#std_x = sm.add_constant(std_x)

lasso_cv = LassoCV(fit_intercept=False)
lasso_cv = lasso_cv.fit(std_x,std_y)
print(f"Optimal Alpha = {lasso_cv.alpha_}")
lasso_beta = lasso_cv.coef_  * (y_scale / x_scale)
print(lasso_beta)
print(lasso_cv.intercept_)

In [ ]:
# Hybrid stepwise function
# requires x: x dataframe , y series, p = number of columns, requires xval function too
# input x, y, p
def hybrid_stepwise(x, y, p,random=False,fold=5):
    included = []

    for i in range(p):
        excluded = [col for col in x if col not in included]
        best_sse = np.inf
        for col in excluded:
            try_x = x[included + [col]]
            beta = lstsq(try_x, y, rcond=None)[0]
            resid = y - try_x @ beta
            sse = resid @ resid
            if sse < best_sse:
                best_sse = sse
                next_var = col
        included.append(next_var)
    #print(included)
    print()
    #print(back_stepwise(x[included],y,p))

    end_models = {}
    for i in range(1,p+1):
        x_col = included[:i]
        #print(x_col)
        #print(x[x_col])
        regressors = x[x_col]
        #x = x[x_col]
        #print(x.columns)
        #print(x.shape[1])
        #print(back_stepwise(regressors,y,p))
        end_models[i] = back_stepwise(regressors,y,p)
    #return end_models 


    hsr_sse_sv = {}

    for i in range(1,p+1):
        hsr_sse_sv[i] = xval_5fold(y,x[end_models[i]],random=random, fold=fold)
    hsr_sse_sv = pd.Series(hsr_sse_sv)
    hybrid_step_model = end_models[hsr_sse_sv.idxmin()]

    return hybrid_step_model, hsr_sse_sv